# Merge join concat

In [12]:
import pandas as pd
import seaborn as sns
import numpy as np

In [5]:
df1 = pd.DataFrame({"ID":[1,2,3,5,9],
                   "Col_1": [1,2,3,4,5],
                   "Col_2": [6,7,8,9,10],
                    "Col_3": [11,12,13,14,15],
                    "Col_4": ["apple","orange","banana","strawberry","rasberry"]                   
                   })
df2 = pd.DataFrame ({"ID":[1,1,3,5],
                   "Col_A": [8,9,10,11],
                   "Col_B": [12,13,15,17],
                    "Col_4": ["apple","orange","banana","kiwi"]
                     })

In [6]:
df1

,ID,Col_1,Col_2,Col_3,Col_4
0,1,1,6,11,apple
1,2,2,7,12,orange
2,3,3,8,13,banana
3,5,4,9,14,strawberry
4,9,5,10,15,rasberry


In [7]:
df2

,ID,Col_A,Col_B,Col_4
0,1,8,12,apple
1,1,9,13,orange
2,3,10,15,banana
3,5,11,17,kiwi


In [8]:
inner = pd.merge(df1,df2)
inner

,ID,Col_1,Col_2,Col_3,Col_4,Col_A,Col_B
0,1,1,6,11,apple,8,12
1,3,3,8,13,banana,10,15


In [9]:
# It's a good idea yo always specify on
pd.merge(df1, df2, on ='ID')

,ID,Col_1,Col_2,Col_3,Col_4_x,Col_A,Col_B,Col_4_y
0,1,1,6,11,apple,8,12,apple
1,1,1,6,11,apple,9,13,orange
2,3,3,8,13,banana,10,15,banana
3,5,4,9,14,strawberry,11,17,kiwi


In [10]:
# this is the sam as the first merge because we are
# specifiying all cloumns that share a name between
# the two DataFrames
pd.merge(df1,df2, on = ["ID","Col_4"])

,ID,Col_1,Col_2,Col_3,Col_4,Col_A,Col_B
0,1,1,6,11,apple,8,12
1,3,3,8,13,banana,10,15


In [11]:
# Suffixes & merginf on cloumns that are unique to 
# each Data Frame
pd.merge(df1, df2, suffixes=['_l','_r'], left_on='Col_2', right_on= 'Col_A')

,ID_l,Col_1,Col_2,Col_3,Col_4_l,ID_r,Col_A,Col_B,Col_4_r
0,3,3,8,13,banana,1,8,12,apple
1,5,4,9,14,strawberry,1,9,13,orange
2,9,5,10,15,rasberry,3,10,15,banana


In [13]:
tips = sns.load_dataset('tips')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


# Merge
Merging is for doing complex column-wise combinations of dataframes in a SQL-like way. If you don't know SQL joins then check out this resource sql joins and comment below (and maybe I'll make a video).

Two merge we need two dataframes, let's make them below:

In [14]:

tips_bill = tips.groupby(['sex', 'smoker'])[['total_bill', 'tip']].sum()
tips_tip = tips.groupby(['sex', 'smoker'])[['total_bill', 'tip']].sum()

del tips_bill['tip']
del tips_tip['total_bill']

In [16]:
tips_bill

total_bill
sex    smoker            
Male   Yes        1337.07
       No         1919.75
Female Yes         593.27
       No          977.68

In [17]:
tips_tip

tip
sex    smoker        
Male   Yes     183.07
       No      302.00
Female Yes      96.74
       No      149.77

Now that we have two datasets that we want to combine (aka take the tips and combine with the total bill), how do we do it? We merge!

Notice that there are a ton of options:

In [21]:
# we can merge on the indexes
pd.merge(tips_bill, tips_tip, 
         right_index=True, left_index=True)

total_bill     tip
sex    smoker                    
Male   Yes        1337.07  183.07
       No         1919.75  302.00
Female Yes         593.27   96.74
       No          977.68  149.77

In [22]:
#we can reset indexes and then merge on the columns - perhaps the easiest way
pd.merge(
    tips_bill.reset_index(), 
    tips_tip.reset_index(),
    on=['sex', 'smoker']
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,96.74
3,Female,No,977.68,149.77


In [23]:
# it can actually infer the above - but be very careful with this
pd.merge(
    tips_bill.reset_index(), 
    tips_tip.reset_index()
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,96.74
3,Female,No,977.68,149.77


In [26]:
# it can merge on partial column and index
pd.merge(
    tips_bill.reset_index(), 
    tips_tip,
    left_on=['sex', 'smoker'],
    right_index=True
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,96.74
3,Female,No,977.68,149.77


In [27]:
#it can do interesting combinations
tips_bill_strange = tips_bill.reset_index(level=0)
tips_bill_strange

,sex,total_bill
smoker,,
Yes,Male,1337.07
No,Male,1919.75
Yes,Female,593.27
No,Female,977.68


In [28]:
pd.merge(
    tips_tip.reset_index(), 
    tips_bill_strange,
    on=['sex', 'smoker']
)

,sex,smoker,tip,total_bill
0,Male,Yes,183.07,1337.07
1,Male,No,302.00,1919.75
2,Female,Yes,96.74,593.27
3,Female,No,149.77,977.68


In [29]:
# we can do any SQL-like functionality
pd.merge(
    tips_bill.reset_index(), 
    tips_tip.reset_index().head(2),
    how='left'
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,NaN
3,Female,No,977.68,NaN


In [30]:
pd.merge(
    tips_bill.reset_index(), 
    tips_tip.reset_index().head(2),
    how='inner'
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00


In [31]:
# and if you add an indicator...
pd.merge(
    tips_bill.reset_index().tail(3), 
    tips_tip.reset_index().head(3),
    how='outer',
    indicator=True
)

,sex,smoker,total_bill,tip,_merge
0,Male,No,1919.75,302.00,both
1,Female,Yes,593.27,96.74,both
2,Female,No,977.68,NaN,left_only
3,Male,Yes,NaN,183.07,right_only


In [32]:

# it can handle columns with the same name
pd.merge(tips_bill, 
         tips_bill, 
         right_index=True, 
         left_index=True,
         suffixes=('_left', '_right')
)

total_bill_left  total_bill_right
sex    smoker                                   
Male   Yes             1337.07           1337.07
       No              1919.75           1919.75
Female Yes              593.27            593.27
       No               977.68            977.68

This is one of the most complex parts of pandas - but it is very important to master. So please do check out the excerises below!

One thing to be careful with here is merging two data types. Strings are not equal to ints!

# Contatenation
Concatenating is for combining more than two dataframes in either column-wise or row-wise. The problem with concatenate is that the combinations it allows you to do are rather simplistic. That's why we need merge.

Concatenate can take as many data frames as you want, but it requires that they are specifically constructed. All of the dataframes you pass in will need to have the same index. So no more using columns as an index.

Let's check out basic use below:

In [33]:
# this adds the dataframes together row wise
pd.concat([tips_bill, tips_bill, tips_tip], sort=False)

total_bill     tip
sex    smoker                    
Male   Yes        1337.07     NaN
       No         1919.75     NaN
Female Yes         593.27     NaN
       No          977.68     NaN
Male   Yes        1337.07     NaN
       No         1919.75     NaN
Female Yes         593.27     NaN
       No          977.68     NaN
Male   Yes            NaN  183.07
       No             NaN  302.00
Female Yes            NaN   96.74
       No             NaN  149.77

In [34]:
# this does it column wise
pd.concat([tips_bill, tips_tip], axis=1)

total_bill     tip
sex    smoker                    
Male   Yes        1337.07  183.07
       No         1919.75  302.00
Female Yes         593.27   96.74
       No          977.68  149.77

In [35]:
# and finally this will add on the dataset where it's from
pd.concat([tips_bill, tips_tip], sort=False, keys=['num0', 'num1'])

total_bill     tip
     sex    smoker                    
num0 Male   Yes        1337.07     NaN
            No         1919.75     NaN
     Female Yes         593.27     NaN
            No          977.68     NaN
num1 Male   Yes            NaN  183.07
            No             NaN  302.00
     Female Yes            NaN   96.74
            No             NaN  149.77


As you can see there is not a ton of functionality to concat, but it is invaluable if you have more than one dataframe or you are looking to append the rows of one dataframe onto another.